In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_04 = pd.read_csv(folderpath + 'data_op_04.csv')
data_op_04.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
4,1,5,42.0041,0.8405,40.0,1354.48,1124.32,3.91,2211.80,130.44,2387.89,0.02,4,1
6,1,7,41.9998,0.8400,40.0,1354.43,1131.44,3.91,2211.82,130.16,2387.88,0.02,4,1
9,1,10,42.0066,0.8400,40.0,1353.19,1125.09,3.91,2211.84,130.32,2387.91,0.02,4,1
12,1,13,42.0029,0.8409,40.0,1350.90,1123.53,3.91,2211.78,130.60,2387.88,0.02,4,1
20,1,21,42.0030,0.8404,40.0,1349.16,1117.33,3.91,2211.82,130.48,2387.90,0.02,4,1


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_04 = tf_remove_sensor_15.fit_transform(data_op_04)
data_op_04.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
4,1,5,42.0041,0.8405,40.0,1354.48,1124.32,3.91,2211.80,130.44,2387.89,4,1
6,1,7,41.9998,0.8400,40.0,1354.43,1131.44,3.91,2211.82,130.16,2387.88,4,1
9,1,10,42.0066,0.8400,40.0,1353.19,1125.09,3.91,2211.84,130.32,2387.91,4,1
12,1,13,42.0029,0.8409,40.0,1350.90,1123.53,3.91,2211.78,130.60,2387.88,4,1
20,1,21,42.0030,0.8404,40.0,1349.16,1117.33,3.91,2211.82,130.48,2387.90,4,1


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_04)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
4,1354.48,1124.32,3.91,2211.80,130.44,2387.89,1
6,1354.43,1131.44,3.91,2211.82,130.16,2387.88,1
9,1353.19,1125.09,3.91,2211.84,130.32,2387.91,1
12,1350.90,1123.53,3.91,2211.78,130.60,2387.88,1
20,1349.16,1117.33,3.91,2211.82,130.48,2387.90,1


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[-0.03471011, -0.51398323,  1.        , -0.50007728, -0.27185   ,
        -0.42928696],
       [-0.04353727,  0.4286228 ,  1.        , -0.43380467, -1.08895102,
        -0.45999999],
       [-0.26245095, -0.41204409,  1.        , -0.36753205, -0.62203615,
        -0.36786091],
       ...,
       [ 1.87019199,  2.54551472,  1.        , -2.05748362, -0.94304012,
        -1.96493836],
       [ 1.91962411,  1.85444681,  1.        , -2.19002884, -0.85549359,
        -2.30278167],
       [ 0.80033965,  2.9056114 ,  1.        , -2.35571037, -1.96441641,
        -2.14921653]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 1, ..., 4, 4, 4])

# Classification steps

## Load Experiment model

In [7]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [9]:
# MLP:
mlp_clf = MLPClassifier(solver='adam', learning_rate='adaptive', 
                        max_iter=1300, learning_rate_init=5e-04, tol=1e-4)

## Put all clf in a dictionary:

In [10]:
classifiers = {'MLP': mlp_clf}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

### Hyperparameter tunning

In [11]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

In [12]:
param_dist_dict = {'MLP': {"hidden_layer_sizes": list(np.arange(2,1001))}
                  }

In [13]:
for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=10, n_iter=50, verbose=5, n_jobs=100, scoring='accuracy')
    random_search[clf].fit(data_with_features_std, labels)
    print('Elapsed time:')
    print(time() - start)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=819 ..........................................
[CV] hidden_layer_sizes=720 ..........................................
[CV] hidden_layer_sizes=720 ..........................................
[CV] hidden_layer_sizes=720 ..........................................
[CV] hidden_la

[Parallel(n_jobs=100)]: Done  88 tasks      | elapsed: 37.1min


[CV] hidden_layer_sizes=350 ..........................................
[CV] . hidden_layer_sizes=372, score=0.6260794473229706, total=23.3min
[CV] hidden_layer_sizes=350 ..........................................
[CV] . hidden_layer_sizes=169, score=0.6228373702422145, total=10.1min
[CV] hidden_layer_sizes=350 ..........................................
[CV] . hidden_layer_sizes=435, score=0.6473638720829732, total=36.9min
[CV] hidden_layer_sizes=18 ...........................................
[CV] . hidden_layer_sizes=372, score=0.6091458153580673, total=24.2min
[CV] hidden_layer_sizes=18 ...........................................
[CV] . hidden_layer_sizes=169, score=0.6231633535004322, total=10.9min
[CV] hidden_layer_sizes=18 ...........................................
[CV] .. hidden_layer_sizes=621, score=0.651685393258427, total=37.6min
[CV] . hidden_layer_sizes=470, score=0.6248919619706137, total=15.7min
[CV] hidden_layer_sizes=18 ...........................................
[CV] h

[Parallel(n_jobs=100)]: Done 250 tasks      | elapsed: 64.6min


[CV] hidden_layer_sizes=817 ..........................................
[CV] . hidden_layer_sizes=177, score=0.6222990492653414, total=14.2min
[CV] hidden_layer_sizes=923 ..........................................
[CV] . hidden_layer_sizes=480, score=0.5953408110440035, total=18.4min
[CV] hidden_layer_sizes=923 ..........................................
[CV] .. hidden_layer_sizes=57, score=0.6326707000864304, total= 6.9min
[CV] hidden_layer_sizes=923 ..........................................
[CV] .. hidden_layer_sizes=57, score=0.6121212121212121, total= 6.1min
[CV] hidden_layer_sizes=923 ..........................................
[CV] .. hidden_layer_sizes=57, score=0.6141868512110726, total= 7.4min
[CV] hidden_layer_sizes=923 ..........................................
[CV] .. hidden_layer_sizes=57, score=0.6179775280898876, total= 7.9min
[CV] hidden_layer_sizes=923 ..........................................
[CV] .. hidden_layer_sizes=480, score=0.615916955017301, total=19.3min
[CV] h

[Parallel(n_jobs=100)]: Done 402 out of 500 | elapsed: 108.2min remaining: 26.4min


[CV] .. hidden_layer_sizes=29, score=0.6277056277056277, total= 6.4min
[CV] . hidden_layer_sizes=301, score=0.6108714408973253, total=14.0min
[CV] ... hidden_layer_sizes=29, score=0.606926406926407, total= 6.7min
[CV] .. hidden_layer_sizes=29, score=0.6186367558239861, total= 9.3min
[CV] . hidden_layer_sizes=687, score=0.6133217993079585, total=17.6min
[CV] . hidden_layer_sizes=602, score=0.6364421416234888, total=22.7min
[CV] . hidden_layer_sizes=125, score=0.6188418323249784, total= 6.8min
[CV] .. hidden_layer_sizes=29, score=0.6226252158894646, total= 9.5min
[CV] . hidden_layer_sizes=125, score=0.6169111302847282, total= 7.7min
[CV] . hidden_layer_sizes=703, score=0.6082830025884383, total=32.0min
[CV] .. hidden_layer_sizes=29, score=0.6065573770491803, total=10.3min
[CV] . hidden_layer_sizes=703, score=0.6228373702422145, total=29.9min
[CV] . hidden_layer_sizes=923, score=0.6404494382022472, total=42.8min
[CV] .. hidden_layer_sizes=29, score=0.6141868512110726, total= 8.6min
[CV] .

[Parallel(n_jobs=100)]: Done 500 out of 500 | elapsed: 118.6min finished


Elapsed time:
7123.287585735321


## Savel results, models and pipeline to a .pkl file 

In [14]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [15]:
experiment.save_models(random_search, name='clf_mlp')
experiment.save_pipeline(data_preprocessing)